In [4]:
from bs4 import BeautifulSoup
import requests
import csv
from urllib.request import Request, urlopen
import urllib
import json
import re
import pandas as pd
import time
from selenium import webdriver
import sys
import numpy as np

In [3]:
header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,'referer':'https://www.google.com/'}

## Defined functions

In [4]:
#parser: object data collection
def object_parser(object_request, subcat):
    
    item_soup = BeautifulSoup(object_request.text, 'lxml')
    
    try:
        name = item_soup.find('span', class_="_oqoid").text
    except:
        name = None
        pass

    try:
        region = item_soup.find('div', class_="_1p8iqzw").text
    except:
        region = None
        pass

    try:
        type_ = item_soup.find('span', class_="_btwk9a2").text
    except:
        type_ = None
        pass
        
    try:
        subtype = item_soup.find('div', class_="_11eqcnu").text
    except:
        subtype = None
        pass

    try:
        rating = item_soup.find('span', class_="_1n8h0vx").text
    except:
        rating = None
        pass

    try:
        ratingcount = item_soup.find('span', class_="_gg5kmr").text
    except:
        ratingcount = None
        pass

    try:
        coord_link = str(item_soup.find_all('meta')[1])
        coord = re.findall("([0-9]+[,.]+[0-9]+)",coord_link)
    except:
        coord = None
        pass
    
    try:
        address = item_soup.find('span', class_="_er2xx9").find('a', class_="_ke2cp9k").text
    except:
        address = item_soup.find('span', class_="_er2xx9").text.replace(u'\u200b', '')
        pass
    
    object_info = pd.DataFrame()
    object_info = object_info.append({'Name':name ,'Region':region , 'Address':address, 'Type' :type_ , 'Subtype' :subtype, \
        'Coord' : coord, 'Rating' : rating, 'RatingCount' : ratingcount, 'Subcategory': subcat}, ignore_index = True)
    return object_info

In [43]:
#function for parsing object from a page with page links
def get_df(cat_dict, cat_name):
    i = 0
    object_info = pd.DataFrame()
    exception_links = []
    for subcat, links in cat_dict.items():

            for link in links:
                i = i+1
                try:
                    #request            
                    time.sleep(1.5)
                    object_request = requests.get(link,headers=header)
                    object_info = object_info.append(object_parser(object_request, str(subcat)))
                except:
                    print('Exception occured on iteration', i)
                    exception_links.append(link)
                    continue
                if (i%100 == 0):
                    print(i)
                    object_info.to_csv(str(subcat)+'temp')
    object_info['Category'] = str(cat_name)
    print("End of the loop")
    return object_info, exception_links

## Parsing the objects from 2gis

### Categories: Subcategories
#### Поесть: 
'Кафе', "Рестораны", "Бары", "Быстрое питание", "Кофейни",
                  "Готовые блюда", "Пиццерии", "Столовые", "Суши-бары"

#### Развлечения:
Кинотеатры', "Караоке", "Антикафе", "Музеи", "Библиотеки","Театры", "Художественные выставки", "Парки культуры и отдыха", "Концертные залы"

#### Медицина: 
"Аптеки", "Стоматологии", "Мед.центры", "Анализы", "Больницы","Диагностические центры", "Детские специалисты"

#### Продукты: 
"Супермаркеты", "Гипермаркеты", "Продуктовые магазины"

#### Спорт: 
"Фитнес-клубы","Тренажёрные залы","Секции","Бассейны","Стадионы"

#### Образование: 
"Гимназии","Лицеи","Лицей-интернаты","Начальные школы","Школы","Школы-интернаты","Детские сады","Частные детские сады","Развитие детей","Колледжи", "Университеты", "Институты","Академии"

#### Государственные учреждения:
'Суды', "Судебная и внесудебная экспертиза","Отделения полиции","Участковые пункты полиции","ОМВД, УМВД, ГУМВД, МВД","Прокуратуры","Следственный комитет", "Цон", "Социальные службы", "Службы занятости", "Налоговые инспекции", "Законодательная власть","Акимат города", "Маслихат города", "Акиматы поселений города", "Посольства, консульства","Государственные службы"


In [6]:
# razvl_dict = np.load('razvl_links.npy',allow_pickle='TRUE').item()
# df_razvl, exc_razvl = get_df(razvl_dict, 'Развлечения')
# df_razvl.to_csv('df_razvl')

In [7]:
# sport_dict = np.load('sport_links.npy',allow_pickle='TRUE').item()
# df_sport, exc_sport = get_df(sport_dict, 'Спорт')
# df_sport.to_csv('df_sport')

In [8]:
# med_dict = np.load('med_links.npy',allow_pickle='TRUE').item()
# df_med, exc_med = get_df(med_dict, 'Медицина')
# df_med.to_csv('df_med')

In [2]:
# poest_dict = np.load('poest_links.npy',allow_pickle='TRUE').item()
# df_poest, exc_poest = get_df(poest_dict, 'Поесть')
# df_poest.to_csv('df_poest')

In [ ]:
# prod_dict = np.load('prod_links.npy',allow_pickle='TRUE').item()
# df_prod, exc_prod = get_df(prod_dict, 'Продукты')
# df_prod.to_csv('df_prod')

In [ ]:
# edu_dict = np.load('edu_links.npy',allow_pickle='TRUE').item()
# df_edu, exc_dict = get_df(edu_dict, 'Образование')
# df_edu.to_csv('df_edu')

In [ ]:
# gov_dict = np.load('gov_links.npy',allow_pickle='TRUE').item()
# df_gov, exc_gov = get_df(gov_dict, 'Государственные учреждения')
# df_gov.to_csv('df_gov')

In [34]:
df_edu = pd.read_csv('df_edu')
df_razvl = pd.read_csv('df_razvl')
df_sport = pd.read_csv('df_sport')
df_gov = pd.read_csv('df_gov')
df_prod = pd.read_csv('df_prod')
df_poest = pd.read_csv('df_poest')
df_med = pd.read_csv('df_med')

gis2020 = pd.concat([df_edu,df_sport,df_razvl,df_gov, df_poest, df_prod, df_med])
gis2020 = gis2020.drop(['Unnamed: 0'], axis= 1)[['Name','Address','Coord','Rating','RatingCount','Region',
                                                  'Type', 'Subtype','Category','Subcategory']]
gis2020.to_csv('gis2020.csv')